<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 63.2 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=e740e4d813a2f85b4c95d69ba2b46cf070e9040dcf9e4d8208efbc7a30b7c09d
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta
 

In [3]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [25]:
ticker_yahoo = yf.Ticker('HINDZINC.NS')
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)


In [5]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [29]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

#print (ath_list)

def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ticker))
    return results

results = paralel_func(ath_list)


In [30]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)

# ticker_list.to_csv('Watchlist.csv', index=False)
ticker_list


,Ticker,Last_price,ATH,two_EMA
0,360ONE.NS,1838.800049,1960.000000,1701.202249
1,3MINDIA.NS,22995.449219,25209.000000,22206.125703
2,ABBOTINDIA.NS,20978.949219,22499.000000,19260.241035
3,AEGISCHEM.NS,362.049988,388.799988,277.581000
4,AGARIND.NS,686.150024,746.200012,594.227999
5,AKZOINDIA.NS,2222.500000,2400.000000,2044.508752
6,ANANDRATHI.NS,802.599976,862.500000,681.024500
7,ARMANFIN.NS,1610.199951,1705.000000,1326.531252
8,ASTRAZEN.NS,3271.649902,3620.000000,3100.373000
9,ATFL.NS,882.299988,969.799988,808.872999


In [31]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  119 of 119 completed


In [172]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

df1 = pd.DataFrame()


for ticker in tickers:
  test = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  test1 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  test2 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  test3 = ta.ema(close=ax_df[('High', ticker)], length=32).to_frame('EMA200_H')
  test4 = ta.ema(close=ax_df[('Close', ticker)], length=32).to_frame('EMA200_C')
  test5 = ta.ema(close=ax_df[('Low', ticker)], length=32).to_frame('EMA200_L')
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
  test2.columns = pd.MultiIndex.from_product([test2.columns, [ticker]])
  test3.columns = pd.MultiIndex.from_product([test3.columns, [ticker]])
  test4.columns = pd.MultiIndex.from_product([test4.columns, [ticker]])
  test5.columns = pd.MultiIndex.from_product([test5.columns, [ticker]])
  df1 = pd.concat([df1, test, test1, test2, test3, test4, test5], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Low','High','Volume'], level=0, axis=1)


In [173]:
ax_df= ax_df.swaplevel(0, 1, 1).sort_index(axis=1)

In [174]:
ax_df= ax_df.tail(1)

In [175]:
import datetime

df2= pd.DataFrame()
for ticker in tickers:
  cond1 = (ax_df.loc[:,(ticker,'Close')] > ax_df.loc[:,(ticker,'EMA200_H')]).bool()
  cond2 = (ax_df.loc[:,(ticker,'Close')] > ax_df.loc[:,(ticker,'Open')]).bool()
  cond3 = (ax_df.loc[:,(ticker,'SUPERTd_10_1.0')] + ax_df.loc[:,(ticker,'SUPERTd_11_2.0')] + ax_df.loc[:,(ticker,'SUPERTd_12_3.0')] > -1).bool()

  if cond1 and cond2 and cond3:
    ax_df1 = ax_df.loc[:,(ticker)]
    ax_df1.columns = pd.MultiIndex.from_product([[ticker],ax_df1.columns])
    df2 = pd.concat([df2,ax_df1], axis=1)
    
#df2.to_csv('test.csv')


In [176]:
pd.set_option('display.max_rows', None)
df2 = df2.T
df2

Date                            2023-02-07
AEGISCHEM.NS  Close             364.950012
              EMA200_C          351.823792
              EMA200_H          360.184027
              EMA200_L          344.404311
              Open              360.450012
              SUPERT_10_1.0     373.721433
              SUPERT_11_2.0     344.162467
              SUPERT_12_3.0     328.873587
              SUPERTd_10_1.0     -1.000000
              SUPERTd_11_2.0      1.000000
              SUPERTd_12_3.0      1.000000
              SUPERTl_10_1.0           NaN
              SUPERTl_11_2.0    344.162467
              SUPERTl_12_3.0    328.873587
              SUPERTs_10_1.0    373.721433
              SUPERTs_11_2.0           NaN
              SUPERTs_12_3.0           NaN
AGARIND.NS    Close             681.650024
              EMA200_C          621.083307
              EMA200_H          632.930558
              EMA200_L          607.949088
              Open              680.000000
              SUPERT_10_1.0     649.308156
              SUPERT_11_2.0     623.275568
              SUPERT_12_3.0     597.762668
              SUPERTd_10_1.0      1.000000
              SUPERTd_11_2.0      1.000000
              SUPERTd_12_3.0      1.000000
              SUPERTl_10_1.0    649.308156
              SUPERTl_11_2.0    623.275568
              SUPERTl_12_3.0    597.762668
              SUPERTs_10_1.0           NaN
              SUPERTs_11_2.0           NaN
              SUPERTs_12_3.0           NaN
ARMANFIN.NS   Close            1585.500000
              EMA200_C         1486.537136
              EMA200_H         1517.122453
              EMA200_L         1456.649595
              Open             1535.000000
              SUPERT_10_1.0    1496.600773
              SUPERT_11_2.0    1435.068094
              SUPERT_12_3.0    1386.018766
              SUPERTd_10_1.0      1.000000
              SUPERTd_11_2.0      1.000000
              SUPERTd_12_3.0      1.000000
              SUPERTl_10_1.0   1496.600773
              SUPERTl_11_2.0   1435.068094
              SUPERTl_12_3.0   1386.018766
              SUPERTs_10_1.0           NaN
              SUPERTs_11_2.0           NaN
              SUPERTs_12_3.0           NaN
ATULAUTO.NS   Close             346.350006
              EMA200_C          314.733296
              EMA200_H          323.790917
              EMA200_L          305.274103
              Open              338.000000
              SUPERT_10_1.0     328.786427
              SUPERT_11_2.0     319.569983
              SUPERT_12_3.0     304.080759
              SUPERTd_10_1.0      1.000000
              SUPERTd_11_2.0      1.000000
              SUPERTd_12_3.0      1.000000
              SUPERTl_10_1.0    328.786427
              SUPERTl_11_2.0    319.569983
              SUPERTl_12_3.0    304.080759
              SUPERTs_10_1.0           NaN
              SUPERTs_11_2.0           NaN
              SUPERTs_12_3.0           NaN
BAJAJ-AUTO.NS Close            3879.449951
              EMA200_C         3706.965485
              EMA200_H         3742.654035
              EMA200_L         3664.585478
              Open             3873.000000
              SUPERT_10_1.0    3893.056731
              SUPERT_11_2.0    3750.051113
              SUPERT_12_3.0    3673.353006
              SUPERTd_10_1.0     -1.000000
              SUPERTd_11_2.0      1.000000
              SUPERTd_12_3.0      1.000000
              SUPERTl_10_1.0           NaN
              SUPERTl_11_2.0   3750.051113
              SUPERTl_12_3.0   3673.353006
              SUPERTs_10_1.0   3893.056731
              SUPERTs_11_2.0           NaN
              SUPERTs_12_3.0           NaN
BAJAJELEC.NS  Close            1176.000000
              EMA200_C         1144.734141
              EMA200_H         1162.156131
              EMA200_L         1127.707806
              Open             1160.500000
              SUPERT_10_1.0    1129.871703
              SUPERT_11_2.0    1093.971649
 